In [1]:
! pip install pennylane
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RNN
import pennylane as qml
from google.colab import files

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.1/19.1 MB 20.3 MB/s eta 0:00:00


In [2]:
upload = files.upload()
predata = pd.read_csv('data.csv')
predata = predata[['Year', 'Value']]
data = predata.groupby('Year')['Value'].mean().reset_index()

Saving data.csv to data.csv


In [3]:
train = data[data['Year'] < 2011]
test = data[data['Year'] >= 2011]

scaler = MinMaxScaler()
train = scaler.fit_transform(train)
test = scaler.transform(test)

x_train = train[:, :-1]
y_train = train[:, -1]
x_test = test[:, :-1]
y_test = test[:, -1]

x_train = np.reshape(x_train, (x_train.shape[0], 1, x_train.shape[1]))
x_test = np.reshape(x_test, (x_test.shape[0], 1, x_test.shape[1]))

In [4]:
n_qubits = 6
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev, interface='tf', diff_method='backprop')
def quantum_circuit(inputs, weights):
    qml.templates.AngleEmbedding(inputs, wires=range(n_qubits))
    qml.templates.BasicEntanglerLayers(weights, wires=range(n_qubits))
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

weight_shapes = {"weights": (8, n_qubits)}

In [5]:
class QuantumRNNCell(tf.keras.layers.AbstractRNNCell):
    def __init__(self, **kwargs):
        super(QuantumRNNCell, self).__init__(**kwargs)
        self.q_weights_real = self.add_weight(shape=(8, n_qubits), initializer="random_normal", trainable=True, dtype=tf.float64, regularizer=regularizers.l2(0.01))
        self.q_weights_imag = self.add_weight(shape=(8, n_qubits), initializer="random_normal", trainable=True, dtype=tf.float64, regularizer=regularizers.l2(0.01))
        self._state_size = n_qubits
        self._output_size = n_qubits

    @property
    def state_size(self):
        return self._state_size

    @property
    def output_size(self):
        return self._output_size

    def call(self, inputs, states):
        states = states[0]
        inputs = tf.cast(inputs, tf.complex128)
        inputs = tf.reshape(inputs, [-1, inputs.shape[-1]])
        weights = tf.complex(self.q_weights_real, self.q_weights_imag)

        def circuit_map(input):
            result = quantum_circuit(input, weights)
            return result

        outputs = tf.vectorized_map(circuit_map, inputs)
        outputs = tf.math.real(outputs)
        outputs = tf.reshape(outputs, [-1, self.output_size])
        new_states = outputs
        return outputs, [new_states]

    def get_initial_state(self, inputs=None, batch_size=None, dtype=tf.float64):
        return tf.zeros((batch_size, self.state_size), dtype=tf.float64)

quantum_rnn_cell = QuantumRNNCell()

In [6]:
model = Sequential()
model.add(RNN(quantum_rnn_cell, return_sequences=True, input_shape=(x_train.shape[1], x_train.shape[2])))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100, return_sequences=False))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(x_train, y_train, batch_size=24, epochs=50)

Cause: Unable to locate the source code of <function _gcd_import at 0x7a7adfcbf400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: Unable to locate the source code of <function _gcd_import at 0x7a7adfcbf400>. Note that functions defined in certain environments, like the interactive Python shell, do not expose their source code. If that is the case, you should define them in a .py source file. If you are certain the code is graph-compatible, wrap the call using @tf.autograph.experimental.do_not_convert. Original error: could not get source code
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Epoch 1/50
3/3 [==============================] - 103s 84ms/step - loss: 0.2577
Epoch 2/50
3/3 [==============================] - 0s 72ms/step - loss: 0.2163
Epoch 3/50
3/3 [==============================] - 0s 67ms/step - loss: 0.1758
Epoch 4/50
3/3 [==============================] - 0s 69ms/step - loss: 0.1412
Epoch 5/50
3/3 [==============================] - 0s 68ms/step - loss: 0.1150
Epoch 6/50
3/3 [==============================] - 0s 63ms/step - loss: 0.0977
Epoch 7/50
3/3 [==============================] - 0s 69ms/step - loss: 0.0913
Epoch 8/50
3/3 [==============================] - 0s 68ms/step - loss: 0.0947
Epoch 9/50
3/3 [==============================] - 0s 86ms/step - loss: 0.0941
Epoch 10/50
3/3 [==============================] - 0s 93ms/step - loss: 0.0985
Epoch 11/50
3/3 [==============================] - 0s 109ms/step - loss: 0.0952

In [7]:
y_pred = model.predict(x_test)

1/1 [==============================] - 39s 39s/step


In [8]:
test_df = pd.DataFrame(test, columns=['Year', 'Value'])
test_df['Value'] = y_test
y_test_inverse = scaler.inverse_transform(test_df)[:, -1]

pred_df = pd.DataFrame(test, columns=['Year', 'Value'])
pred_df['Value'] = y_pred
y_pred_inverse = scaler.inverse_transform(pred_df)[:, -1]

In [9]:
print(f"predictions: {y_pred_inverse}")

predictions: [346.02544298 346.02544298 346.05617132 346.06360362 346.05220371
 346.04544788 342.05793116 335.93023862 337.27283054 346.07327674
 345.24208281 336.68649314 334.46014489]


In [12]:
#GRAPHING THE PREDICTIONS AGAINST REAL MEASURED VALUES FROM data.csv
#Year = range(2011, 2024)
#plt.plot(data['Year'], data['Value'], label='Real Data')
#plt.plot(data['Year'][data['Year'] >= 2011], y_pred_inverse, label='Predicted Values')
#plt.title("Actual vs Predicted Values by the QRNN Model")
#plt.xlabel("Years")
#plt.ylabel("CO2 Emissions (ppm)")
#plt.grid(True)
#plt.legend()
#plt.xlim(data['Year'].min(), data['Year'].max())
#plt.ylim(data['Value'].min(), data['Value'].max())
#plt.show()

In [11]:
rmse = np.sqrt(np.mean((y_pred_inverse - y_test_inverse)**2))
print('Test RMSE: %.2f' % rmse)

Test RMSE: 64.90
